## Starter Code for Assignment 2 - Big Data with H2O

**Multiclass Classification**

Here is some starter code for Assignment 2 - Big Data with H2O.   You will still need to extend this code somewhat.  


[H2O Docs](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html)  
[http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html)  

In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool

Set up some parameters for the analysis.  

In [2]:
data_path=None
all_variables=None
test_path=None
target=None
nthreads=1 
min_mem_size=6 
run_time=333
classification=False
scale=False
max_models=9    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0 

The next sections contains helper functions for automating analysis.             

In [3]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(run_id,analysis,target,run_time,classification,scale,model,balance,balance_threshold,name,nthreads,min_mem_size):
  m_data={}
  m_data['run_id'] =run_id
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [4]:
data_path='data/multiclass_iris_flowers.csv'

In [5]:
data_path = os.path.join(os.path.abspath(os.curdir),data_path)

In [6]:
all_variables=None

In [7]:
run_id=alphabet(9)
# run_id to std out
print (run_id) 

7jPMYu5Da


In [8]:
server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir) 

In [9]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:21480..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /Users/bear/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpe5nccqvt
  JVM stdout: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpe5nccqvt/h2o_bear_started_from_python.out
  JVM stderr: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpe5nccqvt/h2o_bear_started_from_python.err
  Server is running at http://127.0.0.1:21480
Connecting to H2O server at http://127.0.0.1:21480... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.1
H2O cluster version age:,3 months and 25 days !!!
H2O cluster name:,H2O_from_python_bear_n5ax9f
H2O cluster total nodes:,1
H2O cluster free memory:,5.750 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [10]:
# meta data
meta_data = set_meta_data(run_id,analysis,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,nthreads,min_mem_size)
print(meta_data)  

{'run_id': '7jPMYu5Da', 'start_time': 1538542354.297025, 'target': None, 'max_models': 9, 'run_time': 333, 'scale': False, 'classification': False, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1538542354.297031, 'execution_time': 0.0, 'nthreads': 1, 'min_mem_size': 6, 'analysis': 0}


In [11]:
print(data_path)

/Users/bear/Downloads/AutoML/data/multiclass_iris_flowers.csv


In [12]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
df.head()

ID,sepal_length,sepal_width,petal_length,petal_width,species
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
10,4.9,3.1,1.5,0.1,Iris-setosa


In [14]:
df.describe()

Rows:150
Cols:6




,ID,sepal_length,sepal_width,petal_length,petal_width,species
type,int,real,real,real,real,enum
mins,1.0,4.3,2.0,1.0,0.1,
mean,75.5,5.843333333333333,3.053999999999999,3.758666666666667,1.1986666666666665,
maxs,150.0,7.9,4.4,6.9,2.5,
sigma,43.445367992456916,0.8280661279778637,0.43359431136217375,1.764420419952262,0.7631607417008414,
zeros,0,0,0,0,0,
missing,0,0,0,0,0,0
0,1.0,5.1,3.5,1.4,0.2,Iris-setosa
1,2.0,4.9,3.0,1.4,0.2,Iris-setosa
2,3.0,4.7,3.2,1.3,0.2,Iris-setosa


describe()[source]
Generate an in-depth description of this H2OFrame.

The description is a tabular print of the type, min, max, sigma, number of zeros, and number of missing elements for each H2OVec in this H2OFrame.

Returns:	None (print to stdout) 

In [15]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
  target=df.columns[-1]   
y = target

In [16]:
print(y)

species


In [17]:
print(all_variables)

None


In [18]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [19]:
df.describe()

Rows:150
Cols:6




,ID,sepal_length,sepal_width,petal_length,petal_width,species
type,int,real,real,real,real,enum
mins,1.0,4.3,2.0,1.0,0.1,
mean,75.5,5.843333333333333,3.053999999999999,3.758666666666667,1.1986666666666665,
maxs,150.0,7.9,4.4,6.9,2.5,
sigma,43.445367992456916,0.8280661279778637,0.43359431136217375,1.764420419952262,0.7631607417008414,
zeros,0,0,0,0,0,
missing,0,0,0,0,0,0
0,1.0,5.1,3.5,1.4,0.2,Iris-setosa
1,2.0,4.9,3.0,1.4,0.2,Iris-setosa
2,3.0,4.7,3.2,1.3,0.2,Iris-setosa


In [20]:
# independent variables

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

In [21]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [22]:
print(classification)

False


In [23]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [24]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok, val   

In [25]:
ok,val=check_y(y,df)

In [26]:
print(val)

enum


In [27]:
print(ok)

True


In [28]:
if val=='enum':
    print(df[y].levels())

[['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']]


In [29]:
df.describe()

Rows:150
Cols:6




,ID,sepal_length,sepal_width,petal_length,petal_width,species
type,int,real,real,real,real,enum
mins,1.0,4.3,2.0,1.0,0.1,
mean,75.5,5.843333333333333,3.053999999999999,3.758666666666667,1.1986666666666665,
maxs,150.0,7.9,4.4,6.9,2.5,
sigma,43.445367992456916,0.8280661279778637,0.43359431136217375,1.764420419952262,0.7631607417008414,
zeros,0,0,0,0,0,
missing,0,0,0,0,0,0
0,1.0,5.1,3.5,1.4,0.2,Iris-setosa
1,2.0,4.9,3.0,1.4,0.2,Iris-setosa
2,3.0,4.7,3.2,1.3,0.2,Iris-setosa


In [30]:
allV=get_variables_types(df)
allV

{'ID': 'int',
 'petal_length': 'real',
 'petal_width': 'real',
 'sepal_length': 'real',
 'sepal_width': 'real',
 'species': 'enum'}

In [31]:
meta_data['variables']=allV

In [32]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.9])

In [33]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [34]:
model_start_time = time.time()

In [35]:
aml.train(x=X,y=y,training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [37]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [38]:
aml_leaderboard_df

,model_id,mean_per_class_error,logloss,rmse,mse
0,DRF_0_AutoML_20181003_005235,0.000000,2.969224e-02,6.992901e-02,4.890066e-03
1,GBM_grid_0_AutoML_20181003_005235_model_1,0.000000,2.637156e-07,2.265790e-06,5.133806e-12
2,GBM_grid_0_AutoML_20181003_005235_model_72,0.000000,8.353590e-09,1.019511e-08,1.039402e-16
3,GBM_grid_0_AutoML_20181003_005235_model_57,0.000000,1.040738e+00,6.468061e-01,4.183581e-01
4,StackedEnsemble_AllModels_0_AutoML_20181003_00...,0.000000,1.110656e-02,1.458604e-02,2.127524e-04
5,GBM_grid_0_AutoML_20181003_005235_model_3,0.000000,1.034146e-06,8.727223e-06,7.616442e-11
6,GLM_grid_0_AutoML_20181003_005235_model_0,0.000000,7.728962e-03,2.986393e-02,8.918545e-04
7,GBM_grid_0_AutoML_20181003_005235_model_49,0.000000,1.056636e+00,6.523663e-01,4.255818e-01
8,GBM_grid_0_AutoML_20181003_005235_model_13,0.000000,7.541721e-07,7.562790e-06,5.719579e-11
9,GBM_grid_0_AutoML_20181003_005235_model_16,0.000000,2.323549e-05,2.112606e-04,4.463106e-08


In [39]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [40]:
mod_best._id

'DRF_0_AutoML_20181003_005235'

In [41]:
# Get stacked ensemble  
se=get_stacked_ensemble(model_set)

In [42]:
print(se)

StackedEnsemble_BestOfFamily_0_AutoML_20181003_005235


In [43]:
if se is not None:
  mod_best=h2o.get_model(se)

In [44]:
mod_best._id

'StackedEnsemble_BestOfFamily_0_AutoML_20181003_005235'

In [45]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics>

In [46]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [47]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_0_AutoML_20181003_005235
No model summary for this model


ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.43867762915885744
RMSE: 0.6623274334940819

ModelMetricsMultinomialGLM: stackedensemble
** Reported on validation data. **

MSE: 0.45802891296701026
RMSE: 0.6767783337009321

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.4386776291588575
RMSE: 0.662327433494082
<bound method ModelBase.coef_norm of >


In [48]:
bm=stackedensemble_df(aml_leaderboard_df)

In [49]:
bm

['GBM_grid_0_AutoML_20181003_005235_model_1',
 'GLM_grid_0_AutoML_20181003_005235_model_0',
 'DRF_0_AutoML_20181003_005235',
 'XRT_0_AutoML_20181003_005235',
 'DeepLearning_0_AutoML_20181003_005235']

In [50]:
aml_leaderboard_df

,model_id,mean_per_class_error,logloss,rmse,mse
0,DRF_0_AutoML_20181003_005235,0.000000,2.969224e-02,6.992901e-02,4.890066e-03
1,GBM_grid_0_AutoML_20181003_005235_model_1,0.000000,2.637156e-07,2.265790e-06,5.133806e-12
2,GBM_grid_0_AutoML_20181003_005235_model_72,0.000000,8.353590e-09,1.019511e-08,1.039402e-16
3,GBM_grid_0_AutoML_20181003_005235_model_57,0.000000,1.040738e+00,6.468061e-01,4.183581e-01
4,StackedEnsemble_AllModels_0_AutoML_20181003_00...,0.000000,1.110656e-02,1.458604e-02,2.127524e-04
5,GBM_grid_0_AutoML_20181003_005235_model_3,0.000000,1.034146e-06,8.727223e-06,7.616442e-11
6,GLM_grid_0_AutoML_20181003_005235_model_0,0.000000,7.728962e-03,2.986393e-02,8.918545e-04
7,GBM_grid_0_AutoML_20181003_005235_model_49,0.000000,1.056636e+00,6.523663e-01,4.255818e-01
8,GBM_grid_0_AutoML_20181003_005235_model_13,0.000000,7.541721e-07,7.562790e-06,5.719579e-11
9,GBM_grid_0_AutoML_20181003_005235_model_16,0.000000,2.323549e-05,2.112606e-04,4.463106e-08


In [51]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [52]:
bm

['GBM_grid_0_AutoML_20181003_005235_model_1',
 'GLM_grid_0_AutoML_20181003_005235_model_0',
 'DRF_0_AutoML_20181003_005235',
 'XRT_0_AutoML_20181003_005235',
 'DeepLearning_0_AutoML_20181003_005235',
 'StackedEnsemble_BestOfFamily_0_AutoML_20181003_005235']

In [53]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]


In [54]:
aml_leaderboard_df

,model_id,mean_per_class_error,logloss,rmse,mse
0,DRF_0_AutoML_20181003_005235,0.000000,2.969224e-02,0.069929,4.890066e-03
1,GBM_grid_0_AutoML_20181003_005235_model_1,0.000000,2.637156e-07,0.000002,5.133806e-12
6,GLM_grid_0_AutoML_20181003_005235_model_0,0.000000,7.728962e-03,0.029864,8.918545e-04
27,XRT_0_AutoML_20181003_005235,0.000000,3.109663e-02,0.079513,6.322275e-03
55,DeepLearning_0_AutoML_20181003_005235,0.011111,1.193865e-01,0.144493,2.087813e-02
66,StackedEnsemble_BestOfFamily_0_AutoML_20181003...,0.666667,1.089925e+00,0.662327,4.386776e-01


In [55]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [56]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

DRF_0_AutoML_20181003_005235


In [57]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

DRF_0_AutoML_20181003_005235
drf


In [58]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [59]:
meta_data['models']=bm

In [60]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass    

In [61]:
print(models_path)

/Users/bear/Downloads/AutoML/7jPMYu5Da/models


In [62]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass        

{'algo': 'gbm', 'model_id': 'GBM_grid_0_AutoML_20181003_005235_model_1', 'varimp': [('ID', 248.91380310058594, 1.0, 0.8241661338303877), ('petal_length', 29.126144409179688, 0.11701297415559485, 0.09643813051781168), ('petal_width', 23.881074905395508, 0.095941143512018, 0.07907144132356626), ('sepal_length', 0.09504469484090805, 0.0003818377834293928, 0.0003146977697193675), ('sepal_width', 0.002898342674598098, 1.1643961236761463e-05, 9.596558514972594e-06)]}


In [63]:
# DeepLearning

mod,mod_id=get_model_by_algo("Dee",best_models)


In [64]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass       

{'algo': 'deeplearning', 'model_id': 'DeepLearning_0_AutoML_20181003_005235', 'varimp': [('petal_width', 1.0, 1.0, 0.25366501309003175), ('petal_length', 0.935508131980896, 0.935508131980896, 0.23730568254476514), ('ID', 0.7488226294517517, 0.7488226294517517, 0.1899501021019906), ('sepal_length', 0.6679701805114746, 0.6679701805114746, 0.16944066458319407), ('sepal_width', 0.5899060964584351, 0.5899060964584351, 0.14963853768001845)]}


In [65]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass     

In [66]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass     

In [67]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass     

In [68]:
predictions_df=predictions_test(mod_best,test,run_id)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [69]:
predictions_df.head()

predict,Iris-setosa,Iris-versicolor,Iris-virginica
Iris-setosa,0.996104,0,0.0038961
Iris-setosa,0.996104,0,0.0038961
Iris-setosa,0.996104,0,0.0038961
Iris-setosa,0.996104,0,0.0038961
Iris-setosa,0.996104,0,0.0038961
Iris-setosa,0.996104,0,0.0038961
Iris-setosa,0.996104,0,0.0038961
Iris-setosa,0.795953,0.198988,0.00505902
Iris-versicolor,0,0.996104,0.0038961
Iris-versicolor,0,0.996104,0.0038961


In [70]:
predictions_df.describe()

Rows:15
Cols:4




,predict,Iris-setosa,Iris-versicolor,Iris-virginica
type,enum,real,real,real
mins,,0.0,0.0,0.003896103903332681
mean,,0.5179120036754152,0.3416709501872676,0.14041704613731712
maxs,,0.9961038960966673,0.9961038960966673,0.995836431118175
sigma,,0.5039564168630077,0.472723749075455,0.34767217792015637
zeros,,7,7,0
missing,0,0,0,0
0,Iris-setosa,0.9961038960966673,0.0,0.003896103903332681
1,Iris-setosa,0.9961038960966673,0.0,0.003896103903332681
2,Iris-setosa,0.9961038960966673,0.0,0.003896103903332681


In [71]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)    


In [72]:
meta_data

{'X': ['ID', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
 'analysis': 0,
 'balance': False,
 'balance_threshold': 0.2,
 'classification': False,
 'end_time': 1538542706.807482,
 'execution_time': 352.5104570388794,
 'max_models': 9,
 'min_mem_size': 6,
 'mod_best': 'DRF_0_AutoML_20181003_005235',
 'mod_best_algo': 'drf',
 'model_execution_time': 349.95365834236145,
 'models': ['GBM_grid_0_AutoML_20181003_005235_model_1',
  'GLM_grid_0_AutoML_20181003_005235_model_0',
  'DRF_0_AutoML_20181003_005235',
  'XRT_0_AutoML_20181003_005235',
  'DeepLearning_0_AutoML_20181003_005235',
  'StackedEnsemble_BestOfFamily_0_AutoML_20181003_005235'],
 'nthreads': 1,
 'project': None,
 'run_id': '7jPMYu5Da',
 'run_time': 333,
 'scale': False,
 'start_time': 1538542354.297025,
 'target': None,
 'variables': {'ID': 'int',
  'petal_length': 'real',
  'petal_width': 'real',
  'sepal_length': 'real',
  'sepal_width': 'real',
  'species': 'enum'}}

In [73]:
# Clean up
os.chdir(server_path)

In [74]:
h2o.cluster().shutdown()

H2O session _sid_a8f6 closed.


**When availble for your models you will need to make the following plots: **

## Variable Importance Plots

The Variable Importance Plot graphs the VIP values for each X variable. 

https://www.researchgate.net/figure/Random-forest-variable-importance-plot-Variables-are-ranked-in-terms-of-importance-on_fig2_295097543



## Partial Dependence Plots

For models that include only numerical values, you can view a Partial Dependence Plot (PDP) for that model. This provides a graphical representation of the marginal effect of a variable on the class probability (classification) or response (regression).

https://www.kaggle.com/dansbecker/partial-dependence-plots



## Gains/Lift Charts

[Score: Gains/Lift Table](http://h2o-release.s3.amazonaws.com/h2o/master/1648/docs-website/userguide/scoregainslift.html)  

The Gains/Lift Table page uses predicted data to evaluate model performance. The accuracy of the classification model for a random sample is evaluated according to the results when the model is and is not used.

The Gains/Lift Table is particularly useful for direct marketing applications, for example. The gains/lift chart shows the effectiveness of the current model(s) compared to a baseline, allowing users to quickly identify the most useful model.

To create a Gains/Lift table, H2O applies the model to each entry in the original dataset to find the response probability (Pi^), then orders the entries according to their predicted response probabilities. Finally, H2O divides the dataset into equal groups and calculates the average response rate for each group.

H2O uses the response rate of the top ten groups to evaluate the model performance; the highest response and greatest variation rates indicate the best model.

The lift is calculated from the gains. H2O uses the following formula to calculate the lift: λk=rkr⎯⎯⎯

where λk is the lift for k, rk is the response rate for k, and r⎯⎯⎯ is the average response rate for k. In other words, λk defines how much more likely k customers are to respond in comparison to the average response rate.

Requirements:

The vactual column must contain actual binary class labels.
The vpredict column must contain probabilities.

Evaluating Classifiers: Gains and Lift Charts https://youtu.be/1dYOcDaDJLY 

Understanding And Interpreting Gain And Lift Charts

   https://www.datasciencecentral.com/profiles/blogs/understanding-and-interpreting-gain-and-lift-charts
   
   

**Multiclass Classification Metrics**

misclassification  

This is the overall error, the number shown in the bottom right of a confusion matrix.

mean per class error  

The right column in a confusion matrix has an error rate for each class. This is the average of them,  If your classes are balanced (each exactly the same size) it is identical to misclassification.

logloss

Multiclass Classification givea a probability for the target belonging to each class. The confidence assigned to the correct category is used to calculate logloss (and MSE). Logloss disproportionately punishes low numbers, which is another way of saying having high confidence in the wrong answer is a bad thing.


MSE

Mean Squared Error. The error is the distance from of the probability it suggested from the actual probability.  


## Confusion Matrix

A confusion matrix, also known as an error matrix, is a specific table layout that allows visualization of the performance of an algorithm, typically a supervised learning one (in unsupervised learning it is usually called a matching matrix). Each row of the matrix represents the instances in a predicted class while each column represents the instances in an actual class.

https://en.wikipedia.org/wiki/Confusion_matrix
